# Sentinel 2 + Landsat animation with hydrograph

In [18]:
# Import modules
import datacube 
import sys
import xarray as xr
import numpy as np
import calendar
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patheffects as PathEffects
import matplotlib.patches as patches
import matplotlib.dates as mdates
import matplotlib.lines as mlines
from skimage import exposure
from datacube.utils import geometry
from datacube.utils.geometry import CRS
from datetime import datetime, timedelta

# Import external dea-notebooks functions using relative link to Scripts directory
sys.path.append('../../10_Scripts')
import DEADataHandling
import DEAPlotting

# Set up datacube instances
dc = datacube.Datacube(app='Time series animation')
#s2dc = datacube.Datacube(config='/g/data/r78/dc_configs/sentinel2.conf')

# Study area name used for output file
study_area = 'macmarsh'

# Thresholds used for cloud masking (Sentinel 2 is more aggressive)
landsat_masked_prop = 0.99
sentinel_masked_prop = 0.90

# Set up analysis data query using a buffer around a lat-long point
lat, lon, buffer = -30.72360833, 147.53250000, 5000
x, y = geometry.point(lon, lat, CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]
query = {'x': (x - buffer, x + buffer),
         'y': (y - buffer, y + buffer),    
         'time': ('2015-01-01', '2018-07-30'),
         'crs': 'EPSG:3577'}

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Extract cloud-free clear Landsat and Sentinel observations
Use the `load_clearlandsat` function to load Landsat observations and PQ data for multiple sensors (i.e. ls5, ls7, ls8), and return a single xarray dataset containing only observations that contain greater than a specified proportion of clear pixels. 

Load in Sentinel data for S2A and S2B.

In [12]:
# Custom mask that includes only cloudy or cloud shadowed pixels with data for all bands
custom_mask = {'cloud_acca': 'no_cloud', 
               'cloud_fmask': 'no_cloud', 
               'cloud_shadow_acca': 'no_cloud_shadow',
               'cloud_shadow_fmask': 'no_cloud_shadow',
               'contiguous': True}

# Load in data
ls578_ds = DEADataHandling.load_clearlandsat(dc=dc, query=query, product='nbart', 
                                        bands_of_interest=['green', 'nir', 'swir1', 'red'], 
                                        masked_prop=landsat_masked_prop, mask_dict=custom_mask)#, apply_mask=False)  

# Remove problematic timesteps
problem_times = ['2015-08-02', '2015-08-18', '2016-03-13', '2016-04-30', '2017-05-03', '2015-04-28','2016-08-22']
times_to_drop = [ls578_ds.time.loc[i].values for i in problem_times]

# Remove timesteps
ls578_ds = ls578_ds.drop(np.concatenate(times_to_drop), dim="time")


Loading ls5 pixel quality
    Skipping ls5; no valid data for query
Ignoring SLC-off observations for ls7
Loading ls7 pixel quality
    Loading 0 filtered ls7 timesteps
Loading ls8 pixel quality
    Loading 59 filtered ls8 timesteps
Combining and sorting ls7, ls8 data
    Replacing invalid -999 values with NaN (data will be coerced to float64)


## Import Sentinel 2

In [17]:
problem_times = ['2015-08-02', '2015-08-18', '2016-03-13', '2016-04-30', '2017-05-03', '2015-04-28','2016-08-22']
times_to_drop = [ls578_ds.time.loc[i].values for i in problem_times]


In [19]:
dc.list_measurements()

aliases  \
product               measurement                                                              
bom_rainfall_grids    rainfall                                                           NaN   
dsm                   elevation                                                          NaN   
                      slope                                                              NaN   
dsm1sv10              elevation                                                          NaN   
gamma_ray             rad_air_dose_rate_unfiltered                                       NaN   
                      rad_k_equiv_conc_unfiltered                                        NaN   
                      rad_u_equiv_conc_unfiltered                                        NaN   
                      rad_th_equiv_conc_unfiltered                                       NaN   
                      rad_air_dose_rate_filtered                                         NaN   
                      rad_k_equiv_conc_filtered                                          NaN   
                      rad_u_equiv_conc_filtered                                          NaN   
                      rad_th_equiv_conc_filtered                                         NaN   
                      rad_ratio_tk                                                       NaN   
                      rad_ratio_u2t                                                      NaN   
                      rad_ratio_uk                                                       NaN   
                      rad_ratio_ut                                                       NaN   
high_tide_comp_20p    blue                                                               NaN   
                      green                                                              NaN   
                      red                                                                NaN   
                      nir                                                                NaN   
                      swir1                                                              NaN   
                      swir2                                                              NaN   
high_tide_comp_count  count_observations                                                 NaN   
item_v2               relative                                                           NaN   
item_v2_conf          stddev                                                             NaN   
low_tide_comp_20p     blue                                                               NaN   
                      green                                                              NaN   
                      red                                                                NaN   
                      nir                                                                NaN   
                      swir1                                                              NaN   
...                                                                                      ...   
s2b_ard_granule       nbart_nir_1                    [nbart_band_08, nbart_B08, nbart_Band8]   
                      nbart_nir_2                   [nbart_band_8A, nbart_B8A, nbart_Band8A]   
                      nbart_swir_2                  [nbart_band_11, nbart_B11, nbart_Band11]   
                      nbart_swir_3                  [nbart_band_12, nbart_B12, nbart_Band12]   
s2b_level1c_granule   B01                                              [band_01, B01, Band1]   
                      B02                                              [band_02, B02, Band2]   
                      B03                                              [band_03, B03, Band3]   
                      B04                                              [band_04, B04, Band4]   
                      B05                                              [band_05, B05, Band5]   
                      B06                            

In [ ]:
# Load Sentinel
s2a_ds = dc.load(product='s2a_ard_granule', group_by='solar_day', 
                          measurements = ['nbart_swir_2', 'nbart_nir_1', 'nbart_green', 'fmask'], like=ls578_ds)

s2b_ds = dc.load(product='s2b_ard_granule', group_by='solar_day', 
                          measurements = ['nbart_swir_2', 'nbart_nir_1', 'nbart_green', 'fmask'], like=ls578_ds)

In [20]:
# Keep only cloud-free Sentinel 2A data
PQmask = s2a_ds.where(s2a_ds.fmask == 1)
threshold = s2a_ds.x.count() * s2a_ds.y.count() * (1.0 - sentinel_masked_prop)
KeepTimeMask = PQmask.nbart_green.isnull().sum(axis=(1, 2)) <= int(threshold)
s2a_ds = s2a_ds.where(KeepTimeMask).dropna(dim='time')

# Keep only cloud-free Sentinel 2B data
PQmask = s2b_ds.where(s2b_ds.fmask == 1)
threshold = s2b_ds.x.count() * s2b_ds.y.count() * (1.0 - sentinel_masked_prop)
KeepTimeMask = PQmask.nbart_green.isnull().sum(axis=(1, 2)) <= int(threshold)
s2b_ds = s2b_ds.where(KeepTimeMask).dropna(dim='time')

/g/data/v10/public/modules/dea/20181015/lib/python3.6/site-packages/datacube/api/query.py:93: FutureWarning: casting an xarray.Dataset to a boolean will change in xarray v0.11 to only include data variables, not coordinates. Cast the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  if like:
/g/data/v10/public/modules/dea/20181015/lib/python3.6/site-packages/datacube/api/query.py:93: FutureWarning: casting an xarray.Dataset to a boolean will change in xarray v0.11 to only include data variables, not coordinates. Cast the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  if like:


AttributeError: 'Dataset' object has no attribute 'pixel_quality'

## Import flow data
This is used to plot a line on the right panel of the animation. Must be imported as a geopandas dataframe with a datetime index column

In [ ]:
# Set area over which index values should be averaged
line_df = pd.read_csv('/g/data/r78/rt1527/dea-notebooks/Enviroflows/raw_data/macmarsh/FlowData.csv', index_col='Date', usecols=[0,1,2,3], parse_dates=True, dayfirst=True)

# Subset to time covered by datacube query
date_min, date_max = query['time']
line_df = line_df[(line_df.index >= date_min) & (line_df.index <= date_max)]

# Give informative name
# line_df['Max discharge (Ml/d)'] = line_df['mean'] * 2
line_df.rename(columns={'mean':'Mean discharge (Ml/d)'}, inplace=True)
line_df.plot()


## Plot data as animation

In [ ]:
# Combine into single dataset
#s2a_ds.rename({'nir1': 'nir'}, inplace=True)
#s2b_ds.rename({'nir1': 'nir'}, inplace=True)
combined_ds = xr.auto_combine([ls578_ds, s2a_ds, s2b_ds])
combined_ds = combined_ds.sortby('time')
combined_ds

# Create animation
DEAPlotting.animated_timeseriesline(ds=combined_ds, # .isel(time=[0,1,2,3,4,5]), 
                        df=line_df, 
                        output_path='{}_sentinel2.mp4'.format(study_area), 
                        bands=['swir1', 'nir', 'green'], 
                        interval=220, 
                        annotation_kwargs={'fontsize':17},
                        reflect_stand=4200)
